In [1]:
# # === SETUP & IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2

# Ensure output directories exist
os.makedirs('output/video', exist_ok=True)

sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

# === CONNECTION ===
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()
blueprints = world.get_blueprint_library()
spawn_points = map.get_spawn_points()

# === CLEANUP OLD ACTORS ===
for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()


In [2]:
# === LABEL SPAWN POINTS ===
print("\U0001F4CD Drawing enlarged label numbers at each spawn point...")
for idx, sp in enumerate(spawn_points):
    loc = sp.location + carla.Location(z=2.5)
    offsets = [(-0.15, 0), (0.15, 0), (0, -0.15), (0, 0.15), (0, 0)]
    for dx, dy in offsets:
        offset_loc = loc + carla.Location(x=dx, y=dy)
        world.debug.draw_string(offset_loc, f"#{idx}", draw_shadow=True,
                                color=carla.Color(r=255, g=255, b=0), life_time=100.0, persistent_lines=True)
print(f"✅ {len(spawn_points)} bold label numbers displayed in simulator.")

# === USER INPUTS ===
start_index = int(input("\nEnter index for START location: "))
spoof_index = int(input("Enter index for SPOOFED GPS location: "))
end_index = int(input("Enter index for FINAL DESTINATION: "))

start_transform = spawn_points[start_index]
spoof_location = spawn_points[spoof_index].location
end_transform = spawn_points[end_index]

print("\n=== Navigation Plan Summary ===")
print(f"Start Location:      {start_transform.location}")
print(f"Spoofed GPS Location:{spoof_location}")
print(f"Destination:         {end_transform.location}")


📍 Drawing enlarged label numbers at each spawn point...
✅ 155 bold label numbers displayed in simulator.

=== Navigation Plan Summary ===
Start Location:      Location(x=-54.344658, y=137.050995, z=0.600000)
Spoofed GPS Location:Location(x=-44.982998, y=114.955147, z=0.600000)
Destination:         Location(x=-66.794197, y=12.998389, z=0.600000)


In [6]:
# === VEHICLE 2 ===
vehicle_bp = blueprints.filter('mini')[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_transform)
print("\U0001F697 Vehicle 2 spawned.")

spectator = world.get_spectator()
spectator.set_transform(carla.Transform(
    vehicle.get_transform().transform(carla.Location(x=-5, z=2)),
    vehicle.get_transform().rotation))

# === SPOOFING SETUP ===
spoofing_active = False
spoofed_latitude = spoof_location.x
spoofed_longitude = spoof_location.y
frame_id = 0
video_fps = 20
video_writers = {}
data_records = []
sensor_data = { 'gnss': None, 'imu': None }
sensor_list = []
affect_steering = True

# === SENSOR CALLBACKS ===
def follow_vehicle():
    while vehicle.is_alive:
        transform = vehicle.get_transform()
        spectator.set_transform(carla.Transform(
            transform.transform(carla.Location(x=-6, z=3)),
            transform.rotation))
        time.sleep(0.05)

follow_thread = threading.Thread(target=follow_vehicle, daemon=True)
follow_thread.start()

def gps_callback(data):
    sensor_data['gnss'] = {
        'latitude': spoofed_latitude if spoofing_active else data.latitude,
        'longitude': spoofed_longitude if spoofing_active else data.longitude
    }

def imu_callback(data):
    sensor_data['imu'] = {
        'accel_x': data.accelerometer.x,
        'accel_y': data.accelerometer.y,
        'accel_z': data.accelerometer.z,
        'gyro_x': data.gyroscope.x,
        'gyro_y': data.gyroscope.y,
        'gyro_z': data.gyroscope.z
    }

def rgb_callback(image):
    global frame_id
    if not (sensor_data['gnss'] and sensor_data['imu']): return

    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    rgb_img = array[:, :, :3][:, :, ::-1]

    if 'rgb' not in video_writers:
        path = 'output/video/rgb_video_vehicle_2.avi'
        video_writers['rgb'] = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
    video_writers['rgb'].write(rgb_img)

    velocity = vehicle.get_velocity()
    speed = 3.6 * (velocity.x**2 + velocity.y**2 + velocity.z**2)**0.5

    record = {
        'frame_id': frame_id,
        'image_num': image.frame,
        'latitude': sensor_data['gnss']['latitude'],
        'longitude': sensor_data['gnss']['longitude'],
        'accel_x': sensor_data['imu']['accel_x'],
        'accel_y': sensor_data['imu']['accel_y'],
        'accel_z': sensor_data['imu']['accel_z'],
        'gyro_x': sensor_data['imu']['gyro_x'],
        'gyro_y': sensor_data['imu']['gyro_y'],
        'gyro_z': sensor_data['imu']['gyro_z'],
        'steering_angle': vehicle.get_control().steer,
        'throttle': vehicle.get_control().throttle,
        'brake': vehicle.get_control().brake,
        'speed': speed,
        'label': 'spoofed' if spoofing_active else 'normal'
    }
    data_records.append(record)
    frame_id += 1

# === SENSOR ATTACHMENT ===
gnss = world.spawn_actor(blueprints.find('sensor.other.gnss'), carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(blueprints.find('sensor.other.imu'), carla.Transform(), attach_to=vehicle)
camera_bp = blueprints.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# === SENSOR LISTEN ===
gnss.listen(gps_callback)
imu.listen(imu_callback)
camera.listen(rgb_callback)
sensor_list += [gnss, imu, camera]


🚗 Vehicle 2 spawned.


In [7]:


# === ROUTE PLANNING ===
grp = GlobalRoutePlanner(map, 2.0)
route_waypoints = grp.trace_route(start_transform.location, end_transform.location)
for i, (waypoint, _) in enumerate(route_waypoints):
    loc = waypoint.transform.location + carla.Location(z=0.5)
    world.debug.draw_point(loc, size=0.1, color=carla.Color(0, 0, 255), life_time=100.0)

agent = BehaviorAgent(vehicle, behavior='normal')
agent.set_destination(end_transform.location)

# === SPOOFING THREAD ===
def spoofing_trigger(delay=10):
    global spoofing_active
    print(f"\U0001F551 Spoofing starts in {delay}s...")
    time.sleep(delay)
    spoofing_active = True
    print("\U0001F6A8 Spoofing activated.")
threading.Thread(target=spoofing_trigger).start()

# === CONTROL LOOP WITH STEERING EFFECT ===
while vehicle.get_location().distance(end_transform.location) > 2.0:
    world.tick()
    control = agent.run_step()

    if spoofing_active and affect_steering:
        drift = 0.4 * np.sin(frame_id / 30.0)
        control.steer = max(-1.0, min(1.0, control.steer + drift))
        control.throttle = max(0.3, control.throttle)
        control.brake = 0.0
        print(f"[Spoofed] steer: {control.steer:.2f}")

    vehicle.apply_control(control)
    


🕑 Spoofing starts in 10s...
🚨 Spoofing activated.
[Spoofed] steer: 0.39
[Spoofed] steer: 0.38
[Spoofed] steer: 0.36
[Spoofed] steer: 0.31
[Spoofed] steer: 0.26
[Spoofed] steer: 0.20
[Spoofed] steer: 0.17
[Spoofed] steer: 0.13
[Spoofed] steer: 0.09
[Spoofed] steer: -0.01
[Spoofed] steer: -0.02
[Spoofed] steer: -0.01
[Spoofed] steer: -0.01
[Spoofed] steer: 0.05
[Spoofed] steer: -0.01
[Spoofed] steer: -0.02
[Spoofed] steer: -0.04
[Spoofed] steer: -0.04
[Spoofed] steer: -0.06
[Spoofed] steer: -0.06
[Spoofed] steer: -0.06
[Spoofed] steer: -0.07
[Spoofed] steer: -0.08
[Spoofed] steer: 0.01
[Spoofed] steer: -0.03
[Spoofed] steer: -0.04
[Spoofed] steer: -0.05
[Spoofed] steer: -0.05
[Spoofed] steer: -0.06
[Spoofed] steer: -0.06
[Spoofed] steer: -0.07
[Spoofed] steer: -0.08
[Spoofed] steer: -0.08
[Spoofed] steer: -0.09
[Spoofed] steer: -0.09
[Spoofed] steer: -0.09
[Spoofed] steer: -0.10
[Spoofed] steer: -0.10
[Spoofed] steer: -0.09
[Spoofed] steer: -0.11
[Spoofed] steer: -0.11
[Spoofed] steer: -

KeyboardInterrupt: 

In [8]:

# === CLEANUP & SAVE ===
try:
    pd.DataFrame(data_records).to_csv('combined_data_vehicle_2.csv', index=False)
    print("\U0001F4BE combined_data_vehicle_2.csv saved.")
except Exception as e:
    print(f"Error saving CSV: {e}")

for s in sensor_list:
    try:
        s.stop()
        s.destroy()
    except:
        pass

try:
    vehicle.destroy()
except:
    pass

for writer in video_writers.values():
    try:
        writer.release()
    except:
        pass

print("✅ Data and videos saved. All actors cleaned up.")

💾 combined_data_vehicle_2.csv saved.
✅ Data and videos saved. All actors cleaned up.
